In [1]:
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
import pandas as pd
import numpy as np
import altair as alt
import project_functions2 as pf

from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [2]:
stock_list = ['AMZN', 'AAPL', 'FB','GOOGL', 'MSFT', 'TSLA']
stock_objects = {}
for stock in stock_list:
    stock_objects[stock] = yf.Ticker(stock)

In [ ]:
combine_df, stock_dfs = pf.end_to_end_lstm_prep(stock_objects, lookback=30, pred_len=5)

In [ ]:
stock_dfs['FB'].columns

In [ ]:
X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split(combine_df, 365, stock_dfs)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_3d = np.reshape(X_train, (X_train.shape[0], 31, int(X_train.shape[1]/31)))
X_test_3d = np.reshape(X_test, (X_test.shape[0], 31, int(X_test.shape[1]/31)))

In [ ]:
model = Sequential()
model.add(LSTM(units=500, activation="tanh", input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model.add(Dense(units = 1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_train_3d, y_train, epochs=200, batch_size=32, verbose=1)

In [ ]:
predictions = model.predict(X_test_3d)
len( predictions[:len(y_test.dropna())])
predictions.shape

In [ ]:
len(y_test.dropna())

In [ ]:
model_train_score = r2_score(y_test.dropna(), predictions[:len(y_test.dropna())])

In [ ]:
model_train_score

In [ ]:
train_preds = model.predict(X_train_3d)
model_score = r2_score(y_train, train_preds)
model_score

In [ ]:
model2 = Sequential()
model2.add(LSTM(units=2500, activation="tanh", input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model2.add(Dense(units = 1))
model2.compile(optimizer = 'adam', loss = 'mean_squared_error')
model2.fit(X_train_3d, y_train, epochs=200, batch_size=32, verbose=1)

In [ ]:
predictions2 = model2.predict(X_test_3d)
model_score2 = r2_score(y_test.dropna(), predictions[:len(y_test.dropna())])
model_score2

In [ ]:
train_preds2 = model2.predict(X_train_3d)
model_train_score2 = r2_score(y_train, train_preds2)
model_train_score2

In [ ]:
stock_dfs = {}
drop_list = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'Mt',
       '5 Day Open Mean', '5 Day High Mean', '5 Day Low Mean',
       '5 Day Close Mean', '5 Day Volume Mean', '5 Day Open Var',
       '5 Day High Var', '5 Day Low Var', '5 Day Close Var',
       '5 Day Volume Var', '5 Day High', '5 Day Low',
       '10 Day Open Mean', '10 Day High Mean', '10 Day Low Mean',
       '10 Day Close Mean', '10 Day Volume Mean', '10 Day Open Var',
       '10 Day High Var', '10 Day Low Var', '10 Day Close Var',
       '10 Day Volume Var', '10 Day High', '10 Day Low', 
             '20 Day Open Mean', '20 Day High Mean', '20 Day Low Mean',
       '20 Day Close Mean', '20 Day Volume Mean', '20 Day Open Var',
       '20 Day High Var', '20 Day Low Var', '20 Day Close Var',
       '20 Day Volume Var', '20 Day High', '20 Day Low']
clost_drop = ['Close', '- 1 Days Close', '- 2 Days Close', '- 3 Days Close',
               '- 4 Days Close', '- 5 Days Close']
for stock in stock_objects:
    stock_dfs[stock] = stock_objects[stock].history(period='max')
    stock_dfs[stock] = pf.rolling_aves(stock_dfs[stock])
    #stock_dfs[stock].drop(drop_list, axis=1, inplace=True)
    stock_dfs[stock] = pf.lstm_prep(stock_dfs[stock], lookback=5)
    stock_dfs[stock] = pf.future_percent_change_setup(stock_dfs[stock], 5)
    #stock_dfs[stock].drop(clost_drop, axis=1, inplace=True)
    
    
combine_df = pf.combiner(stock_dfs)

X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split_scaled(combine_df, 365, stock_dfs)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_3d = np.reshape(X_train, (X_train.shape[0], 6, int(X_train.shape[1]/6)))
X_test_3d = np.reshape(X_test, (X_test.shape[0], 6, int(X_test.shape[1]/6)))

In [ ]:
model2 = Sequential()
model2.add(LSTM(units=500, return_sequences=True, activation="tanh", input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model2.add(Dropout(0.2))
model2.add(LSTM(units=500, return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(units=500))
model2.add(Dropout(0.2))
model2.add(Dense(units = 1))
model2.compile(optimizer = 'adam', loss = 'mean_squared_error')
model2.fit(X_train_3d, y_train, epochs=10, batch_size=32, verbose=1)

In [ ]:
predictions2 = model2.predict(X_test_3d)
model_score2 = r2_score(y_test.dropna(), predictions2[:len(y_test.dropna())])
model_score2

In [ ]:
train_preds2 = model2.predict(X_train_3d)
model_train_score2 = r2_score(y_train, train_preds2)
model_train_score2

In [ ]:
stock_dfs = {}
drop_list = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'Mt',
       '5 Day Open Mean', '5 Day High Mean', '5 Day Low Mean',
       '5 Day Close Mean', '5 Day Volume Mean', '5 Day Open Var',
       '5 Day High Var', '5 Day Low Var', '5 Day Close Var',
       '5 Day Volume Var', '5 Day High', '5 Day Low',
       '10 Day Open Mean', '10 Day High Mean', '10 Day Low Mean',
       '10 Day Close Mean', '10 Day Volume Mean', '10 Day Open Var',
       '10 Day High Var', '10 Day Low Var', '10 Day Close Var',
       '10 Day Volume Var', '10 Day High', '10 Day Low', 
             '20 Day Open Mean', '20 Day High Mean', '20 Day Low Mean',
       '20 Day Close Mean', '20 Day Volume Mean', '20 Day Open Var',
       '20 Day High Var', '20 Day Low Var', '20 Day Close Var',
       '20 Day Volume Var', '20 Day High', '20 Day Low']
clost_drop = ['Close', '- 1 Days Close', '- 2 Days Close', '- 3 Days Close',
               '- 4 Days Close', '- 5 Days Close']
for stock in stock_objects:
    stock_dfs[stock] = stock_objects[stock].history(period='max')
    stock_dfs[stock] = pf.rolling_aves(stock_dfs[stock])
    stock_dfs[stock].drop(drop_list, axis=1, inplace=True)
    stock_dfs[stock] = pf.lstm_prep(stock_dfs[stock], lookback=5)
    stock_dfs[stock] = pf.future_percent_change_setup(stock_dfs[stock], 5)
    stock_dfs[stock].drop(clost_drop, axis=1, inplace=True)
    
    
combine_df = pf.combiner(stock_dfs)

X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split_scaled(combine_df, 365, stock_dfs)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_3d = np.reshape(X_train, (X_train.shape[0], 6, int(X_train.shape[1]/6)))
X_test_3d = np.reshape(X_test, (X_test.shape[0], 6, int(X_test.shape[1]/6)))

In [ ]:
model3 = Sequential()
model3.add(LSTM(units=500, return_sequences=True, activation="tanh", input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model3.add(Dropout(0.2))
model3.add(LSTM(units=500, return_sequences=True))
model3.add(Dropout(0.2))
model3.add(LSTM(units=500))
model3.add(Dropout(0.2))
model3.add(Dense(units = 1))
model3.compile(optimizer = 'adam', loss = 'mean_squared_error')
model3.fit(X_train_3d, y_train, epochs=10, batch_size=32, verbose=1)

In [ ]:
predictions3 = model3.predict(X_test_3d)
model_score3 = r2_score(y_test.dropna(), predictions3[:len(y_test.dropna())])
model_score3

In [ ]:
train_preds3 = model3.predict(X_train_3d)
model_train_score3 = r2_score(y_train, train_preds3)
model_train_score3

# Percentage Change Tests 5 and 30 day lookback

In [8]:
close_drop = ['Close']
for day in range(1,5):
    close_drop.append('Close in ' + str(day) + ' Days')
close_drop

['Close',
 'Close in 1 Days',
 'Close in 2 Days',
 'Close in 3 Days',
 'Close in 4 Days']

In [22]:
stock_dfs = {}
drop_list = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'Mt',
       '5 Day Open Mean', '5 Day High Mean', '5 Day Low Mean',
       '5 Day Close Mean', '5 Day Volume Mean', '5 Day Open Var',
       '5 Day High Var', '5 Day Low Var', '5 Day Close Var',
       '5 Day Volume Var', '5 Day High', '5 Day Low',
       '10 Day Open Mean', '10 Day High Mean', '10 Day Low Mean',
       '10 Day Close Mean', '10 Day Volume Mean', '10 Day Open Var',
       '10 Day High Var', '10 Day Low Var', '10 Day Close Var',
       '10 Day Volume Var', '10 Day High', '10 Day Low', 
             '20 Day Open Mean', '20 Day High Mean', '20 Day Low Mean',
       '20 Day Close Mean', '20 Day Volume Mean', '20 Day Open Var',
       '20 Day High Var', '20 Day Low Var', '20 Day Close Var',
       '20 Day Volume Var', '20 Day High', '20 Day Low']
close_drop = ['Close']
for day in range(1,6):
    close_drop.append('- ' + str(day) + ' Days Close')
for stock in stock_objects:
    stock_dfs[stock] = stock_objects[stock].history(period='max')
    stock_dfs[stock] = pf.rolling_aves(stock_dfs[stock])
    stock_dfs[stock].drop(drop_list, axis=1, inplace=True)
    stock_dfs[stock] = pf.lstm_prep(stock_dfs[stock], lookback=5)
    stock_dfs[stock] = pf.future_percent_change_setup(stock_dfs[stock], 5)
    stock_dfs[stock].drop(close_drop, axis=1, inplace=True)
    
    
combine_df = pf.combiner(stock_dfs)

X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split(combine_df, 365, stock_dfs)
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)
y_train = y_scaler.fit_transform(y_train)
y_test = y_scaler.fit_transform(y_test)
X_train_3d = np.reshape(X_train, (X_train.shape[0], 6, int(X_train.shape[1]/6)))
X_test_3d = np.reshape(X_test, (X_test.shape[0], 6, int(X_test.shape[1]/6)))

In [10]:
model4 = Sequential()
model4.add(LSTM(units=500, return_sequences=True, activation="tanh", input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model4.add(Dropout(0.2))
model4.add(LSTM(units=500, return_sequences=True))
model4.add(Dropout(0.2))
model4.add(LSTM(units=500))
model4.add(Dropout(0.2))
model4.add(Dense(units = 1))
model4.compile(optimizer = 'adam', loss = 'mean_squared_error')
model4.fit(X_train_3d, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
997/997 [==============================] - 177s 174ms/step - loss: 0.0084
Epoch 2/100
997/997 [==============================] - 176s 176ms/step - loss: 0.0026s - loss:
Epoch 3/100
997/997 [==============================] - 175s 176ms/step - loss: 0.0024
Epoch 4/100
997/997 [==============================] - 167s 168ms/step - loss: 0.0025
Epoch 5/100
997/997 [==============================] - 161s 161ms/step - loss: 0.0024
Epoch 6/100
997/997 [==============================] - 161s 161ms/step - loss: 0.0024
Epoch 7/100
997/997 [==============================] - 161s 162ms/step - loss: 0.0023
Epoch 8/100
997/997 [==============================] - 161s 162ms/step - loss: 0.0023
Epoch 9/100
997/997 [==============================] - 161s 161ms/step - loss: 0.0023
Epoch 10/100
997/997 [==============================] - 161s 162ms/step - loss: 0.0024s - l
Epoch 11/100
997/997 [==============================] - 161s 161ms/step - loss: 0.0023
Epoch 12/100
997/997 [================

997/997 [==============================] - 160s 161ms/step - loss: 0.0022
Epoch 96/100
997/997 [==============================] - 160s 161ms/step - loss: 0.0022
Epoch 97/100
997/997 [==============================] - 162s 163ms/step - loss: 0.0023
Epoch 98/100
997/997 [==============================] - 161s 162ms/step - loss: 0.0022
Epoch 99/100
997/997 [==============================] - 160s 161ms/step - loss: 0.0022
Epoch 100/100
997/997 [==============================] - 163s 163ms/step - loss: 0.0021


In [23]:
predictions4 = model4.predict(X_test_3d)
predictions4 = y_scaler.inverse_transform(predictions4)
y_test = y_scaler.inverse_transform(y_test)
model_score4 = r2_score(y_test[:-len(stock_dfs)*5], predictions4[:-len(stock_dfs)*5])
model_score4

-0.0002794508815626706

In [24]:
train_preds4 = model4.predict(X_train_3d)
train_preds4 = y_scaler.inverse_transform(train_preds4)
y_train = y_scaler.inverse_transform(y_train)
model_train_score4 = r2_score(y_train, train_preds4)
model_train_score4

0.020785874447717956

In [25]:
stock_dfs = {}
drop_list = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'Mt',
       '5 Day Open Mean', '5 Day High Mean', '5 Day Low Mean',
       '5 Day Close Mean', '5 Day Volume Mean', '5 Day Open Var',
       '5 Day High Var', '5 Day Low Var', '5 Day Close Var',
       '5 Day Volume Var', '5 Day High', '5 Day Low',
       '10 Day Open Mean', '10 Day High Mean', '10 Day Low Mean',
       '10 Day Close Mean', '10 Day Volume Mean', '10 Day Open Var',
       '10 Day High Var', '10 Day Low Var', '10 Day Close Var',
       '10 Day Volume Var', '10 Day High', '10 Day Low', 
             '20 Day Open Mean', '20 Day High Mean', '20 Day Low Mean',
       '20 Day Close Mean', '20 Day Volume Mean', '20 Day Open Var',
       '20 Day High Var', '20 Day Low Var', '20 Day Close Var',
       '20 Day Volume Var', '20 Day High', '20 Day Low']
close_drop = ['Close']
for day in range(1,31):
    close_drop.append('- ' + str(day) + ' Days Close')
for stock in stock_objects:
    stock_dfs[stock] = stock_objects[stock].history(period='max')
    stock_dfs[stock] = pf.rolling_aves(stock_dfs[stock])
    stock_dfs[stock].drop(drop_list, axis=1, inplace=True)
    stock_dfs[stock] = pf.lstm_prep(stock_dfs[stock], lookback=30)
    stock_dfs[stock] = pf.future_percent_change_setup(stock_dfs[stock], 5)
    stock_dfs[stock].drop(close_drop, axis=1, inplace=True)
    
    
combine_df = pf.combiner(stock_dfs)

X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split(combine_df, 365, stock_dfs)
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)
y_train = y_scaler.fit_transform(y_train)
y_test = y_scaler.fit_transform(y_test)
X_train_3d = np.reshape(X_train, (X_train.shape[0], 31, int(X_train.shape[1]/31)))
X_test_3d = np.reshape(X_test, (X_test.shape[0], 31, int(X_test.shape[1]/31)))

In [26]:
model5 = Sequential()
model5.add(LSTM(units=500, return_sequences=True, activation="tanh", input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model5.add(Dropout(0.2))
model5.add(LSTM(units=500, return_sequences=True))
model5.add(Dropout(0.2))
model5.add(LSTM(units=500))
model5.add(Dropout(0.2))
model5.add(Dense(units = 1))
model5.compile(optimizer = 'adam', loss = 'mean_squared_error')
model5.fit(X_train_3d, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
997/997 [==============================] - 968s 968ms/step - loss: 0.0429
Epoch 2/100
997/997 [==============================] - 905s 908ms/step - loss: 0.0027
Epoch 3/100
997/997 [==============================] - 984s 987ms/step - loss: 0.0026
Epoch 4/100
997/997 [==============================] - 945s 948ms/step - loss: 0.0025
Epoch 5/100
997/997 [==============================] - 944s 947ms/step - loss: 0.0024
Epoch 6/100
997/997 [==============================] - 912s 915ms/step - loss: 0.0025
Epoch 7/100
997/997 [==============================] - 919s 922ms/step - loss: 0.0024
Epoch 8/100
997/997 [==============================] - 947s 950ms/step - loss: 0.0024
Epoch 9/100
997/997 [==============================] - 906s 909ms/step - loss: 0.0023
Epoch 10/100
997/997 [==============================] - 892s 894ms/step - loss: 0.0024
Epoch 11/100
997/997 [==============================] - 841s 844ms/step - loss: 0.0023
Epoch 12/100
997/997 [==============================

In [27]:
predictions5 = model5.predict(X_test_3d)
predictions5 = y_scaler.inverse_transform(predictions5)
y_test = y_scaler.inverse_transform(y_test)
model_score5 = r2_score(y_test[:-len(stock_dfs)*5], predictions5[:-len(stock_dfs)*5])
model_score5

-0.007025678225370147

In [28]:
train_preds5 = model5.predict(X_train_3d)
train_preds5 = y_scaler.inverse_transform(train_preds5)
y_train = y_scaler.inverse_transform(y_train)
model_train_score5 = r2_score(y_train, train_preds5)
model_train_score5

-0.0007052570995516039

# Price Change 5 and 30 day lookback

In [29]:
stock_dfs = {}
drop_list = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'Mt',
       '5 Day Open Mean', '5 Day High Mean', '5 Day Low Mean',
       '5 Day Close Mean', '5 Day Volume Mean', '5 Day Open Var',
       '5 Day High Var', '5 Day Low Var', '5 Day Close Var',
       '5 Day Volume Var', '5 Day High', '5 Day Low',
       '10 Day Open Mean', '10 Day High Mean', '10 Day Low Mean',
       '10 Day Close Mean', '10 Day Volume Mean', '10 Day Open Var',
       '10 Day High Var', '10 Day Low Var', '10 Day Close Var',
       '10 Day Volume Var', '10 Day High', '10 Day Low', 
             '20 Day Open Mean', '20 Day High Mean', '20 Day Low Mean',
       '20 Day Close Mean', '20 Day Volume Mean', '20 Day Open Var',
       '20 Day High Var', '20 Day Low Var', '20 Day Close Var',
       '20 Day Volume Var', '20 Day High', '20 Day Low']
close_drop = ['Close']
for day in range(1,6):
    close_drop.append('- ' + str(day) + ' Days Close')
for stock in stock_objects:
    stock_dfs[stock] = stock_objects[stock].history(period='max')
    stock_dfs[stock] = pf.rolling_aves(stock_dfs[stock])
    #stock_dfs[stock].drop(drop_list, axis=1, inplace=True)
    stock_dfs[stock] = pf.lstm_prep(stock_dfs[stock], lookback=5)
    stock_dfs[stock] = pf.future_close_setup(stock_dfs[stock], 5)
    #stock_dfs[stock].drop(close_drop, axis=1, inplace=True)
    
    
combine_df = pf.combiner(stock_dfs)

X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split(combine_df, 365, stock_dfs)
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)
y_train = y_scaler.fit_transform(y_train)
y_test = y_scaler.fit_transform(y_test)
X_train_3d = np.reshape(X_train, (X_train.shape[0], 6, int(X_train.shape[1]/6)))
X_test_3d = np.reshape(X_test, (X_test.shape[0], 6, int(X_test.shape[1]/6)))

In [30]:
model6 = Sequential()
model6.add(LSTM(units=500, return_sequences=True, activation="tanh", input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model6.add(Dropout(0.2))
model6.add(LSTM(units=500, return_sequences=True))
model6.add(Dropout(0.2))
model6.add(LSTM(units=500))
model6.add(Dropout(0.2))
model6.add(Dense(units = 1))
model6.compile(optimizer = 'adam', loss = 'mean_squared_error')
model6.fit(X_train_3d, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
997/997 [==============================] - 192s 189ms/step - loss: 0.0015
Epoch 2/100
997/997 [==============================] - 191s 191ms/step - loss: 2.9206e-04
Epoch 3/100
997/997 [==============================] - 191s 191ms/step - loss: 2.6524e-04
Epoch 4/100
997/997 [==============================] - 189s 190ms/step - loss: 1.9889e-04
Epoch 5/100
997/997 [==============================] - 190s 191ms/step - loss: 2.0845e-04
Epoch 6/100
997/997 [==============================] - 188s 188ms/step - loss: 2.1243e-04s - loss: 2.1247e-
Epoch 7/100
997/997 [==============================] - 187s 188ms/step - loss: 2.0688e-04s - loss: 2.0692e
Epoch 8/100
997/997 [==============================] - 187s 188ms/step - loss: 1.4779e-04
Epoch 9/100
997/997 [==============================] - 188s 188ms/step - loss: 2.2380e-04
Epoch 10/100
997/997 [==============================] - 189s 190ms/step - loss: 1.4860e-04
Epoch 11/100
997/997 [==============================] - 191s 192ms/s

In [31]:
predictions6 = model6.predict(X_test_3d)
predictions6 = y_scaler.inverse_transform(predictions6)
y_test = y_scaler.inverse_transform(y_test)
model_score6 = r2_score(y_test[:-len(stock_dfs)*5], predictions6[:-len(stock_dfs)*5])
model_score6

0.597922184049694

In [32]:
train_preds6 = model6.predict(X_train_3d)
train_preds6 = y_scaler.inverse_transform(train_preds6)
y_train = y_scaler.inverse_transform(y_train)
model_train_score6 = r2_score(y_train, train_preds6)
model_train_score6

0.9984521684427576

In [38]:
stock_dfs = {}
drop_list = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'Mt',
       '5 Day Open Mean', '5 Day High Mean', '5 Day Low Mean',
       '5 Day Close Mean', '5 Day Volume Mean', '5 Day Open Var',
       '5 Day High Var', '5 Day Low Var', '5 Day Close Var',
       '5 Day Volume Var', '5 Day High', '5 Day Low',
       '10 Day Open Mean', '10 Day High Mean', '10 Day Low Mean',
       '10 Day Close Mean', '10 Day Volume Mean', '10 Day Open Var',
       '10 Day High Var', '10 Day Low Var', '10 Day Close Var',
       '10 Day Volume Var', '10 Day High', '10 Day Low', 
             '20 Day Open Mean', '20 Day High Mean', '20 Day Low Mean',
       '20 Day Close Mean', '20 Day Volume Mean', '20 Day Open Var',
       '20 Day High Var', '20 Day Low Var', '20 Day Close Var',
       '20 Day Volume Var', '20 Day High', '20 Day Low']
close_drop = ['Close']
for day in range(1,30):
    close_drop.append('- ' + str(day) + ' Days Close')
for stock in stock_objects:
    stock_dfs[stock] = stock_objects[stock].history(period='max')
    stock_dfs[stock] = pf.rolling_aves(stock_dfs[stock])
    #stock_dfs[stock].drop(drop_list, axis=1, inplace=True)
    stock_dfs[stock] = pf.lstm_prep(stock_dfs[stock], lookback=30)
    stock_dfs[stock] = pf.future_close_setup(stock_dfs[stock], 5)
    #stock_dfs[stock].drop(close_drop, axis=1, inplace=True)
    
    
combine_df = pf.combiner(stock_dfs)

X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split(combine_df, 365, stock_dfs)
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)
y_train = y_scaler.fit_transform(y_train)
y_test = y_scaler.fit_transform(y_test)
X_train_3d = np.reshape(X_train, (X_train.shape[0], 31, int(X_train.shape[1]/31)))
X_test_3d = np.reshape(X_test, (X_test.shape[0], 31, int(X_test.shape[1]/31)))

In [34]:
model7 = Sequential()
model7.add(LSTM(units=500, return_sequences=True, activation="tanh", input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model7.add(Dropout(0.2))
model7.add(LSTM(units=500, return_sequences=True))
model7.add(Dropout(0.2))
model7.add(LSTM(units=500))
model7.add(Dropout(0.2))
model7.add(Dense(units = 1))
model7.compile(optimizer = 'adam', loss = 'mean_squared_error')
model7.fit(X_train_3d, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
997/997 [==============================] - 790s 789ms/step - loss: 0.0170
Epoch 2/100
997/997 [==============================] - 788s 790ms/step - loss: 2.4065e-04
Epoch 3/100
997/997 [==============================] - 789s 791ms/step - loss: 2.0584e-04
Epoch 4/100
997/997 [==============================] - 789s 791ms/step - loss: 2.0944e-04
Epoch 5/100
997/997 [==============================] - 788s 791ms/step - loss: 2.2622e-04
Epoch 6/100
997/997 [==============================] - 789s 791ms/step - loss: 1.8847e-04
Epoch 7/100
997/997 [==============================] - 789s 791ms/step - loss: 1.8591e-04
Epoch 8/100
997/997 [==============================] - 791s 793ms/step - loss: 1.6722e-04
Epoch 9/100
997/997 [==============================] - 789s 792ms/step - loss: 1.7976e-04
Epoch 10/100
997/997 [==============================] - 790s 793ms/step - loss: 1.8773e-04
Epoch 11/100
997/997 [==============================] - 791s 793ms/step - loss: 1.7115e-04
Epoch 12/100

In [35]:
predictions7 = model7.predict(X_test_3d)
predictions7 = y_scaler.inverse_transform(predictions7)
y_test = y_scaler.inverse_transform(y_test)
model_score7 = r2_score(y_test[:-len(stock_dfs)*5], predictions7[:-len(stock_dfs)*5])
model_score7

0.5751908966353667

In [39]:
train_preds7 = model7.predict(X_train_3d)
train_preds7 = y_scaler.inverse_transform(train_preds7)
y_train = y_scaler.inverse_transform(y_train)
model_train_score7 = r2_score(y_train, train_preds7)
model_train_score7

0.9984745680192805